In [174]:
import ccxt
import os
import datetime
import math

from dotenv import dotenv_values, load_dotenv
from pprint import pprint
from arch import arch_model

# import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

from functions.BS_pricer import BS_pricer
from functions.Parameters import Option_param
from functions.Processes import Diffusion_process

import numpy as np
import pandas as pd
import scipy as scp

# Set the float_format to limit the number of decimal places to 3
pd.options.display.float_format = '{:,.3f}'.format

import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [147]:
# Load environment variables from .env file
load_dotenv()

# Set the API keys for the exchanges
delta_api_key = os.getenv('DELTA_API_KEY')
delta_api_secret = os.getenv('DELTA_API_SECRET')
deribit_api_key = os.getenv('DERIBIT_API_KEY')
deribit_api_secret = os.getenv('DERIBIT_API_SECRET')

# Create the exchange objects
delta = ccxt.delta({
    'apiKey': delta_api_key,
    'secret': delta_api_secret,
})
deribit = ccxt.deribit({
    'apiKey': deribit_api_key,
    'secret': deribit_api_secret,
})

# Create the dictionary
exchanges_dict = {'delta': delta, 'deribit': deribit}
pprint(exchanges_dict)

{'delta': ccxt.delta(), 'deribit': ccxt.deribit()}


In [148]:
# delta_instruments = exchanges_dict['delta'].fetchMarkets()
# deribit_instruments = exchanges_dict['deribit'].fetchMarkets()
instruments_dict = {k: exchanges_dict[k].fetchMarkets() for k in exchanges_dict}
instruments_dict['delta'] = {d['symbol']: d for d in instruments_dict['delta']}
instruments_dict['deribit'] = {d['symbol']: d for d in instruments_dict['deribit']}

In [149]:
# We define the contract we are looking for
contract_specification = {
    'expiration': '230127',
    'payoff': 'C',
    'underlying': 'BTC',
    'strike': 17000,
    'quote': 'USDT'
}

In [150]:
def get_ticker(strike, expiration, underlying, quote, payoff):
    # Construct the ticker symbol for the contract
    delta_ticker_specification = f"{underlying}/{quote}:{quote}-{expiration}:{strike}:{payoff}"
    if quote == "USDT": quote = "USD"
    deribit_ticker_specification = f"{underlying}/{quote}:{underlying}-{expiration}-{strike}-{payoff}"
    # Fetch the ticker data from the Delta exchange
    delta_ticker = exchanges_dict['delta'].fetch_ticker(delta_ticker_specification)
    pprint(f"Delta ticker: {delta_ticker}")

    # Fetch the ticker data from the Deribit exchange
    deribit_ticker = exchanges_dict['deribit'].fetch_ticker(deribit_ticker_specification)
    pprint(f"Deribit ticker: {deribit_ticker}")

    return {
        "delta": delta_ticker,
        "deribit": deribit_ticker,
    }
    
tickers_dict = get_ticker(**contract_specification)

("Delta ticker: {'symbol': 'BTC/USDT:USDT-230127:17000:C', 'timestamp': "
 "1673185417372, 'datetime': '2023-01-08T13:43:37.372Z', 'high': 467.6, 'low': "
 "433.1, 'bid': None, 'bidVolume': None, 'ask': None, 'askVolume': None, "
 "'vwap': 16937.093425572915, 'open': 467.6, 'close': 433.1, 'last': 433.1, "
 "'previousClose': None, 'change': -34.5, 'percentage': -7.378100940975193, "
 "'average': 450.35, 'baseVolume': 0.192, 'quoteVolume': 3251.92193771, "
 "'info': {'close': '433.1', 'contract_type': 'call_options', 'greeks': "
 "{'delta': '0.49208203', 'gamma': '0.00032480', 'rho': '4.08346658', 'spot': "
 "'16930.94980827388', 'theta': '-12.93884367', 'vega': '15.37844914'}, "
 "'high': '467.6', 'low': '433.1', 'mark_price': '456.84607678', 'mark_vol': "
 "'0.3183646767865866422653198243', 'oi': '4.3760', 'oi_contracts': '4376', "
 "'oi_value': '4.3760', 'oi_value_symbol': 'BTC', 'oi_value_usd': "
 "'74094.0973', 'open': '467.6', 'price_band': {'lower_limit': "
 "'0.10000000000000000

In [151]:
def parse_option_spectification_delta(option_specification):
    expiration = datetime.datetime.strptime(option_specification['expiration'], "%y%m%d")

    return {
        "underlying": option_specification['underlying'],
        "quote": option_specification['quote'],
        "expiration": expiration,
        "strike": float(option_specification['strike']),
        "payoff": "call" if option_specification['payoff'] in ("C", "call", "Call", "CALL") else "put",
    }

option_specification = parse_option_spectification_delta(contract_specification)
pprint(option_specification)

{'expiration': datetime.datetime(2023, 1, 27, 0, 0),
 'payoff': 'call',
 'quote': 'USDT',
 'strike': 17000.0,
 'underlying': 'BTC'}


In [152]:
now = datetime.datetime.now()
forward_date = option_specification["expiration"]
print((now, forward_date))

timedelta_until_expiration = forward_date - now

days_until_expiration = timedelta_until_expiration.total_seconds() / 86400
print(days_until_expiration)

exchanges_option_contract = {}

(datetime.datetime(2023, 1, 8, 14, 43, 40, 676183), datetime.datetime(2023, 1, 27, 0, 0))
18.386334766400463


In [153]:
pprint(tickers_dict['delta']['symbol'])

# pprint(instruments_dict['delta'][tickers_dict['delta']['symbol']])
delta_maker_fees_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['maker'])*100
delta_taker_fees_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['taker'])*100
delta_premium_commission_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['info']['product_specs']['premium_commission_rate'])

pprint([
    delta_maker_fees_rate,
    delta_taker_fees_rate,
    delta_premium_commission_rate
])

'BTC/USDT:USDT-230127:17000:C'
[0.03, 0.03, 0.1]


In [154]:
pprint(tickers_dict['deribit']['symbol'])
# pprint(instruments_dict['deribit'][tickers_dict['deribit']['symbol']])

deribit_maker_fees_rate = float(instruments_dict['deribit'][tickers_dict['deribit']['symbol']]['maker'])*100
deribit_taker_fees_rate = float(instruments_dict['deribit'][tickers_dict['deribit']['symbol']]['taker'])*100
deribit_premium_commission_rate = 0.125

pprint([
    deribit_maker_fees_rate,
    deribit_taker_fees_rate,
    deribit_premium_commission_rate
])

'BTC/USD:BTC-230127-17000-C'
[0.03, 0.03, 0.125]


In [155]:
"""
try:
    a = exchanges_dict['delta'].fetchFees()
    pprint(f"fetchFees: {a}")
    #b = exchanges_dict['delta'].fetchTradingFee(tickers_dict['delta']['info']['product_id'])
    #pprint(f"fetchFees: {b}")
    #c = exchanges_dict['delta'].fetchTradingFees()
    #pprint(f"fetchFees: {c}")
    # d = exchanges_dict['delta'].fetchTransactionFee()
    # pprint(f"fetchFees: {d}")
    #e = exchanges_dict['delta'].fetchTransactionFees()
    #pprint(f"fetchFees: {e}")
    #a = exchanges_dict['deribit'].fetchFees()
    #pprint(f"fetchFees: {a}")
    b = exchanges_dict['deribit'].fetchTradingFee(tickers_dict['deribit']['info']['instrument_name'])
    pprint(f"fetchFees: {b}")
    c = exchanges_dict['deribit'].fetchTradingFees()
    pprint(f"fetchFees: {c}")
    d = exchanges_dict['deribit'].fetchTransactionFee()
    pprint(f"fetchFees: {d}")
    e = exchanges_dict['deribit'].fetchTransactionFees()
    pprint(f"fetchFees: {e}")
except Exception as e:
    pprint(e)
"""

'\ntry:\n    a = exchanges_dict[\'delta\'].fetchFees()\n    pprint(f"fetchFees: {a}")\n    #b = exchanges_dict[\'delta\'].fetchTradingFee(tickers_dict[\'delta\'][\'info\'][\'product_id\'])\n    #pprint(f"fetchFees: {b}")\n    #c = exchanges_dict[\'delta\'].fetchTradingFees()\n    #pprint(f"fetchFees: {c}")\n    # d = exchanges_dict[\'delta\'].fetchTransactionFee()\n    # pprint(f"fetchFees: {d}")\n    #e = exchanges_dict[\'delta\'].fetchTransactionFees()\n    #pprint(f"fetchFees: {e}")\n    #a = exchanges_dict[\'deribit\'].fetchFees()\n    #pprint(f"fetchFees: {a}")\n    b = exchanges_dict[\'deribit\'].fetchTradingFee(tickers_dict[\'deribit\'][\'info\'][\'instrument_name\'])\n    pprint(f"fetchFees: {b}")\n    c = exchanges_dict[\'deribit\'].fetchTradingFees()\n    pprint(f"fetchFees: {c}")\n    d = exchanges_dict[\'deribit\'].fetchTransactionFee()\n    pprint(f"fetchFees: {d}")\n    e = exchanges_dict[\'deribit\'].fetchTransactionFees()\n    pprint(f"fetchFees: {e}")\nexcept Exception

In [156]:
### DeltaExchange Format
"""
    Retrieve
    IV
    index price (Underlying price)
    days until expiration
    strike price
    interest rate
    dividend yield (0)
"""
delta_option_contract = tickers_dict['delta']
greeks = delta_option_contract["info"]["greeks"]
ask_iv = float(delta_option_contract["info"]["quotes"]['ask_iv'])
ask_size = float(delta_option_contract["info"]["quotes"]['ask_size'])
best_ask = float(delta_option_contract["info"]["quotes"]['best_ask'])
best_bid = float(delta_option_contract["info"]["quotes"]['best_bid'])
bid_iv = float(delta_option_contract["info"]["quotes"]['bid_iv'])
bid_size = float(delta_option_contract["info"]["quotes"]['bid_size'])
mark_iv = float(delta_option_contract["info"]["quotes"]['mark_iv'])
strike_price = float(delta_option_contract["info"]["strike_price"])
spot_price = float(delta_option_contract["info"]["spot_price"])

exchanges_option_contract.update(
    {
        'delta': {
            'best_ask': best_ask,
            'spot_price': spot_price,
            'ask_iv': ask_iv,
        }
    }
)

In [157]:
### Deribit Format
deribit_option_contract = tickers_dict['deribit']
greeks = deribit_option_contract["info"]["greeks"] # Deribit format
ask_iv = float(deribit_option_contract['info']['ask_iv']) if float(deribit_option_contract['info']['ask_iv']) else float(deribit_option_contract['info']['mark_iv']) 
strike_price = option_specification['strike']
spot_price = float(deribit_option_contract["info"]["underlying_price"])
best_ask_price = float(deribit_option_contract["info"]['best_ask_price']) if float(deribit_option_contract['info']['best_ask_price']) else float(deribit_option_contract['info']['mark_price']) 

exchanges_option_contract.update(
    {
        'deribit': {
            'best_ask': best_ask_price*spot_price,
            'spot_price': spot_price,
            'ask_iv': ask_iv/100.0,
        }
    }
)

In [158]:
def binomial_price(S0, K, T, r, sigma, N=15000, payoff="call"):
    # N number of periods or number of time steps  

    dT = float(T) / N                             # Delta t
    u = np.exp(sigma * np.sqrt(dT))                 # up factor
    d = 1.0 / u                                   # down factor 

    V = np.zeros(N+1)                             # initialize the price vector
    S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

    a = np.exp(r * dT)    # risk free compounded return
    p = (a - d)/ (u - d)  # risk neutral up probability
    q = 1.0 - p           # risk neutral down probability   

    if payoff =="call":
        V[:] = np.maximum(S_T-K, 0.0)
    else:
        V[:] = np.maximum(K-S_T, 0.0)

    for i in range(N-1, -1, -1):
        V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
            
    return V[0]


In [170]:
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
# since = exchange.milliseconds () - 86400000  # -1 day from now
start_date = '2021-01-01T00:00:00Z' # fetch from a certain starting datetime
t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbols = [
    'BTC/USDT:USDT',
    'ETH/USDT:USDT',
    #'AAVE/USDT:USDT',
    #'BNB/USDT:USDT',
    #'CRV/USDT:USDT',
    #'SNX/USDT:USDT',
    #'LINK/USDT:USDT',
]
exchange_list = ['delta', 'deribit']
data_df = pd.DataFrame(None, columns=header).set_index('Timestamp')

for exchange_name, exch in exchanges_dict.items():
    print(exch)
    if exchange_name == "deribit": continue
    if exch.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print(f'{exchange_name} does not support fetching OHLC data. Please use another exchange')
        print('-'*80)
        quit()
    if (not hasattr(exch, 'timeframes')) or (t_frame not in exch.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print(f'The requested timeframe ({t_frame}) is not available from {exchange_name}\n')
        print('Available timeframes are:')
        for key in exch.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exch.load_markets()
    for coin in symbols:
        print(coin)
        since = exch.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exch.milliseconds():
            coin_ohlcv = exch.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])

data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
data_df.to_csv(f"./data/{start_date}_{t_frame}")

Delta Exchange
BTC/USDT:USDT
ETH/USDT:USDT
Deribit


In [172]:
df = pd.read_csv(f"./data/{start_date}_{t_frame}", index_col="Date")
# option['underlyin']/option['quote']:option['quote']
returns = 100 * df[df["Symbol"] == "BTC/USDT:USDT"]["Close"].pct_change().dropna()
am = arch_model(returns, vol="Garch", p=1, o=0, q=1, dist="Normal")
res = am.fit(update_freq=5)
pprint(t_frame)
pprint(days_until_expiration)
integer_t_frame = 24 if t_frame in (('1h',)) else 1

forecast = res.forecast(horizon=integer_t_frame*int(days_until_expiration)+1, reindex=False)
# forecast = res.forecast(horizon=1, reindex=False)
variance_forecast = forecast.variance.iloc[-1][0]
volatility_forecast = np.sqrt(variance_forecast)
my_annualized_volatility_forecast = volatility_forecast * np.sqrt(252) / 100
# Cette VOL est issu de la prediction du GARCH a comparer avec les IV de l'Option Wheel 
# et trade le mispricing
pprint(my_annualized_volatility_forecast)

Iteration:      5,   Func. Count:     36,   Neg. LLF: 2007.2591878795997
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2007.2532823218557
            Iterations: 8
            Function evaluations: 51
            Gradient evaluations: 8
'1d'
18.386334766400463
0.39937545146029846


In [202]:
def price_range(iv, current_price, day_to_expiration):
    range_delta = float(current_price*iv*math.sqrt(day_to_expiration/365))
    return [round(current_price - range_delta,3), round(current_price + range_delta,3)]
### JUST AN EXAMPLE
price_range(my_annualized_volatility_forecast, option_specification['strike'], days_until_expiration)

[15476.188, 18523.812]

In [203]:
for exch, option  in exchanges_option_contract.items():
   print(f"{'='*20} {exch.upper()} {'='*20}")
   info = {
      "S0": float(option['spot_price']),
      "K": float(option_specification['strike']),
      "T": float(days_until_expiration/365),
      "r": float(1/100),
      "sigma": float(option['ask_iv']),
      "payoff": option_specification["payoff"]
   }
   # pprint(f"{info['sigma']:.03f}")
   # BS CLASSIC WITH IV
   theorical_option_price = BS_pricer.BlackScholes(**info)
   # BS CLASSIC WITH GARCH IV
   my_option_price = BS_pricer.BlackScholes(
      S0=float(option['spot_price']),
      K=float(option_specification['strike']),
      T=float(days_until_expiration/365),
      r=float(1/100),
      sigma=float(my_annualized_volatility_forecast),
      payoff=option_specification["payoff"]     
   )
   # pprint(f"{float(my_annualized_volatility_forecast):.03f}")
   # BIN PRICING WITH THEIR IV   
   bin_price = binomial_price(**info)

   # PRINT
   print(f"Theorical price, Their IV: {theorical_option_price:.03f}, {float(option['ask_iv'])*100:.03f}%")
   print(f"Price Range: {price_range(float(option['ask_iv']), float(option['spot_price']), days_until_expiration)}")
   print(f"GARCH price, My IV: {my_option_price:.03f}, {float(my_annualized_volatility_forecast)*100:.03f}%")
   print(f"Price Range: {price_range(float(my_annualized_volatility_forecast), float(option['spot_price']), days_until_expiration)}")
   print(f"binomial price: {bin_price:.03f}") 
   print(f"current market price {option['best_ask']:.03f}")

print(f"="*(42+len(max(list(exchanges_dict.keys())))))

==================== DELTA ====================
Theorical price, Their IV: 473.645, 33.148%
Price Range: [15671.36, 18190.602]
GARCH price, My IV: 576.573, 39.938%
Price Range: [15413.355, 18448.607]
binomial price: 473.652
current market price 477.000
==================== DERIBIT ====================
Theorical price, Their IV: 464.973, 32.230%
Price Range: [15716.037, 18167.043]
GARCH price, My IV: 581.889, 39.938%
Price Range: [15422.968, 18460.112]
binomial price: 464.970
current market price 465.892


In [ ]:
### DATA RETRIEVE BACKUP

for exch in exchange_list:
    print(exch)
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in symbols:
        print(coin)
        since = exchange.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exchange.milliseconds():
            coin_ohlcv = exchange.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])

data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
data_df.to_csv(f"./data/{start_date}_{t_frame}")

'[1231.36, 1270.64]'